In [3]:
# -*- coding: utf-8 -*-
"""
Sample codes for predictive outcomes of knee replacement surgery replicating TKR work
Any patient related data has been de-identified or removed in this example for the security purpose
Method applied: missing data first imputed either by means or kmean clustering. GLM model, RF, and GBM tested

"""
######################################################
###                import library                  ###
######################################################
# import library for feature selection, predictive model and dependencies for imputation
import random
from sklearn.preprocessing import Imputer
from sklearn.cross_validation import cross_val_score
from sklearn import linear_model
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Imputer
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.cross_validation import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from scipy import interp
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel
# import library for cross validation, sensitivity and specificity
from sklearn.cross_validation import cross_val_predict
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (__config__.py, line 6)

In [ ]:

######################################################
###                  Define functions              ###
######################################################
## function for k-mean imputation
def km_impute(x_mis, centers):
    ## function for imputing missing value in a panda vector using k_mean
    # identify missing value index
    delv = np.where(np.isnan(x_mis)==True)
    keep = np.where(np.isnan(x_mis)==False)
    # deleting the corresponding index in k mean centroid
    kmc = np.delete(centers, delv, axis=1)
    x_keep = x_mis.iloc[keep].values
    # find the cluster closest to the complete vector (x_keep)
    gp = metrics.pairwise_distances_argmin(x_keep, kmc)
    # extract center and fill in the missing value    
    gp_center = centers[gp]
    x_mis.iloc[delv] = gp_center[0,delv]
    return x_mis

## function for getting ROC curve plot with 5-fold CV
def ROC_CV(model, X, Y, outcome, cv):
    mean_tpr = 0.0
    mean_fpr = np.linspace(0, 1, 100)
    for i in range(cv):    
        # 5 fold cv scores
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y) 
        if np.sum(Y)/len(Y)<0.2:
            # balancing sample        
            Y_train.index = range(len(Y_train))
            X_train.index = range(len(Y_train))        
            k1 = Y_train.index[Y_train == 1]
            k2 = random.sample(Y_train.index[Y_train ==0], len(k1))
            k = list(k1) + list(k2)
            Y_train = Y_train[k]
            X_train = X_train.ix[k,:]        
        # training model
        probas_ = model.fit(X_train, Y_train).predict_proba(X_test)
        fpr, tpr, threshold = metrics.roc_curve(Y_test, probas_[:,1])
        mean_tpr += interp(mean_fpr, fpr, tpr)
        mean_tpr[0] = 0.0
        roc_auc = metrics.auc(fpr, tpr)
        plt.plot(fpr, tpr, lw=1, label='ROC fold %d (area=%0.2f)' % (i, roc_auc))
        
    plt.plot([0,1], [0,1], '--', color = (0.6, 0.6, 0.6), label='50/50')
    mean_tpr/=cv
    mean_tpr[-1] = 1.0
    mean_auc = metrics.auc(mean_fpr, mean_tpr)
    plt.plot(mean_fpr, mean_tpr, 'k--', label = 'Mean ROC (area = %0.2f)' % mean_auc, lw=2)
    plt.xlim([-0.05, 1.05])
    plt.ylim([-0.05, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic ' + outcome)
    plt.legend(loc="lower right", prop={'size':8})
    plt.show()

## function for obtaining feature importance value and graph
def feature_importance(model, X, Y):
    # feature importance
    importances = model.feature_importances_
    std = np.std([tree.feature_importances_ for tree in model.estimators_], axis=0)
    indices = np.argsort(importances)[::-1]    
    # print the feature ranking
    print("Feature ranking:")
    for f in range(X.shape[1]):
        print("%d. feature %s (%f)" % (f + 1, X.columns.values[indices[f]], importances[indices[f]]))    
    # feature importnace graph
    plt.figure()
    plt.title("Feature importances")
    xlabel = X.columns.values[indices]
    plt.bar(range(X.shape[1]), importances[indices], color="b", yerr=std[indices], align="center")
    plt.xticks(range(X.shape[1]), xlabel, rotation='vertical')
    plt.xlim([-1, X.shape[1]])
    plt.show()    

## simple missing data imputation ##
def impute(imp_method, X_orig, nc):
    # option 1: simple mean 
    if (imp_method=='mean'):
        print('imputing using mean')
        imputer = Imputer(missing_values=np.nan, strategy="mean", axis=0)
        X_imp = pd.DataFrame(imputer.fit_transform(X_orig))
        X_imp.columns = X_orig.columns
        X_imp.index = X_orig.index
    # option 2: K-mean
    if(imp_method=='kmean'):
        print('imputing using kmean')
        # get k mean centers 
        k_means = KMeans(init='k-means++', n_clusters=nc, n_init=10)
        X_slim_norm = (X_orig-X_orig.mean())/(X_orig.max() - X_orig.min())
        k_means.fit(X_slim_norm.dropna(axis=0))
        k_means_cluster_center = k_means.cluster_centers_
        X_imp = X_slim_norm.transpose().apply(km_impute, args=(k_means_cluster_center,)).transpose()
        # save group id
        group_id = k_means.predict(X_imp)
        # denormalize
        X_imp = ((X_imp)*(X_orig.max() - X_orig.min()))+X_orig.mean()
        # add group id
        X_imp['group_id'] = group_id
    return X_imp


In [ ]:
######################################################
###        Import data, impute missing values      ###
######################################################
# import data from data mart
knee_data=pd.read_csv('/data/new_feb_2017/processed/output/merged_data_year.csv', low_memory=False)

# convert age category
knee_data.ix[(knee_data['AGEC']=='>=80'),'AGEC'] = 80
knee_data['AGEC']=pd.to_numeric(knee_data['AGEC'])
knee_data['SEX'].replace(['F','M'], [1,0], inplace=True)


# subset 
knee_data1 = knee_data.loc[(knee_data['DAYS'] >0) & (knee_data['DAYS'] <=365)]

# set prediction target, change target as necessary:longlos, comp90, rev180, recov_issues, disp_home, recent_op, readmit30
target = 'AE_site_1'
# select comparable explorys features
# Depression: GNMDYN7
# Endocrine: GNMDYN11
# Smoking:GNMDYN17
# Anmeia: GNMDYN18 (hematological)
# hypertenion: 'GNMDYN20'
# Neuro: GNMDYN23
# Rheumatism: GNMDYN21 (inflammatory disorder)
# Respiratory: GNMDYN28

dlist=['SEX', 'AGEC', 'BMI', 'GNMDYN7', 'GNMDYN11','GNMDYN17', 'GNMDYN18', 'GNMDYN20', 'GNMDYN21', 'GNMDYN23', 'GNMDYN28'] #hypertenion

X_orig=knee_data1[dlist]
# Impute to get complete data
#X_imp=impute('kmean', X_orig, 9)
X_imp = X_orig
# Concat X & Y and remove row with missing Y
#X_imp = X_imp.drop(['group_id'], axis=1)
Y = knee_data1[target]
df = pd.concat([X_imp, Y], axis=1)
df = df.dropna()
X = df.drop([target], axis=1)
Y = df[target]

######################################################
### Feature selection based on classification tree ###
######################################################
# feature selection using tree-based method, previoius experiment tried L1 and chi-square
clf = ExtraTreesClassifier()
clf = clf.fit(X, Y)
impt = clf.feature_importances_
model = SelectFromModel(clf, prefit=True)
X_new=pd.DataFrame(model.transform(X))
c=np.array(X.columns.values)
indices = np.argsort(impt)[::-1]
od = sorted(indices[(range(X_new.shape[1]))])
cname = c[od]
X_new.columns=cname

###################################################
### Model accuracy and other evaluation metrics ###
###################################################
# specify predictive models
glm = linear_model.LogisticRegression(C=1e5) 
rf = RandomForestClassifier(n_estimators=10, max_depth=None, min_samples_split=1, random_state=0)
gbm = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
# define feature matrix and label
X = X_new
Y = Y
# run GLM - logistic regression
glm_fit = glm.fit(X, Y)
# run random forest
rf_fit = rf.fit(X, Y)
# run gradient boosting
gbm_fit = gbm.fit(X, Y)

###################################################
### Model accuracy and other evaluation metrics ###
###################################################

## ROC based on cross validation
ROC_CV(glm_fit, X, Y, outcome=target, cv=5)
ROC_CV(rf_fit, X, Y, outcome=target, cv=5)
ROC_CV(gbm_fit, X, Y, outcome=target, cv=5)

## feature importance from random forest
feature_importance(rf_fit, X, Y)

## sensitivity and specificty for each model
# select predictive model
mfit=gbm
# begin calculation of sensitivity and specificity from cross validation
cv = 5
ppv = []
sen = []
for i in range(cv):    
        # 5 fold cv scores
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y)
        # further balacing the training set to have equal number of 0 nad 1
        if np.sum(Y)/len(Y)<0.2:
            Y_train.index = range(len(Y_train))
            X_train.index = range(len(Y_train))        
            k1 = Y_train.index[Y_train == 1]
            k2 = random.sample(Y_train.index[Y_train ==0], len(k1))
            k = list(k1) + list(k2)
            Y_train = Y_train[k]
            X_train = X_train.ix[k,:]
        Y_pred = mfit.fit(X_train, Y_train).predict(X_test)
        ppv.append(precision_score(Y_test, Y_pred))        
        sen.append(recall_score(Y_test, Y_pred))
 
np.mean(ppv)
np.mean(sen)